In [1]:
import pandas as pd

game_reviews = pd.read_csv('data\\Processed Data\\game_reviews.csv')
games = pd.read_csv('data\\Processed Data\\games.csv')

To Do:
 - What data needs to be added to review table for the engine?
 - Preliminary Neural Network model
 - How to allow a user to access the model, and make further preferential selections?

#### Prep Data

In [2]:
titles = games[['title', 'game_ID']]
reviews = pd.merge(game_reviews, titles, how="left", on="game_ID")
reviews = reviews[pd.notnull(reviews['review_text'])]

In [3]:
reviews.isna().sum()

game_ID             0
user_ID             0
score            3170
review_text         0
date           193598
title               0
dtype: int64

#### Build Model

In [4]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess

In [6]:
#assert gensim.models.doc2vec.FAST_VERSION > -1

In [7]:
# MyDocs reading from a data frame
class MyDocs(object):
    def __iter__(self):
        for i in range(reviews.shape[0]):
            yield TaggedDocument(words=simple_preprocess(reviews.iloc[i,3]), tags=['%s' % reviews.iloc[i,5]])

In [8]:
%%time
import multiprocessing
import os
cores = multiprocessing.cpu_count()

if not os.path.exists('models/doc2vec.model'):
    print("start traing doc2vec model...")
    documents = MyDocs()
    doc2vec_model = Doc2Vec(dm=1, dbow_words=1, vector_size=200, window=8, min_count=20, workers=cores)
    doc2vec_model.build_vocab(documents)
    doc2vec_model.train(documents, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)
    if not os.path.exists('models'):
        os.makedirs('models')
        doc2vec_model.save('models/doc2vec.model')
    else:
        doc2vec_model.save('models/doc2vec.model')
else:
    doc2vec_model = Doc2Vec.load('models/doc2vec.model')

start traing doc2vec model...
Wall time: 17min 8s


In [11]:
def search(keyword):
    result = []
    for name in reviews.title:
        if keyword in name.lower():
            result.append(name)
    return set(result)

In [31]:
search('overwatch')

{'Overwatch'}

In [26]:
games[games['title']=='Dwarf Fortress']

,game_ID,title,platform_ID,metascore,average_user_score,summary,developer_ID,publisher_ID,release_date,ESRB_rating,url


In [32]:
print(doc2vec_model.docvecs.most_similar('Overwatch', topn=20))

[('LawBreakers', 0.8211004734039307), ('Heroes of the Storm', 0.7917600274085999), ('Battleborn', 0.7679393291473389), ('Valorant', 0.7619352340698242), ('Bleeding Edge', 0.7391541004180908), ('Paladins: Champions of the Realm', 0.72760009765625), ('Apex Legends', 0.6942335963249207), ('Super Monday Night Combat', 0.6853289008140564), ('League of Legends', 0.6812535524368286), ('Loadout', 0.6728128790855408), ('Dirty Bomb', 0.6688874363899231), ('Arena of Valor', 0.6632523536682129), ('Awesomenauts', 0.6631989479064941), ('Plants vs Zombies: Garden Warfare', 0.6400943994522095), ('Gigantic', 0.6392277479171753), ('SMITE', 0.6377713680267334), ("Tom Clancy's Rainbow Six Siege", 0.6347534656524658), ('Dota 2', 0.6346404552459717), ('Drawn to Death', 0.6341026425361633), ('Team Fortress 2', 0.6339164972305298)]
